# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686555


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:04<01:00,  2.16s/it]

Rendering models:  10%|█         | 3/30 [00:05<00:50,  1.86s/it]

Rendering models:  13%|█▎        | 4/30 [00:05<00:34,  1.33s/it]

Rendering models:  20%|██        | 6/30 [00:05<00:23,  1.03it/s]

...

Rendering models:  80%|████████  | 24/30 [00:08<00:01,  5.96it/s]

Rendering models:  87%|████████▋ | 26/30 [00:08<00:00,  6.77it/s]

Rendering models:  93%|█████████▎| 28/30 [00:08<00:00,  8.09it/s]

Rendering models: 100%|██████████| 30/30 [00:09<00:00,  7.57it/s]

not-logged-in-ec4ead1d8579ea060634    0.000405
shirogane                             0.000742
cschwefl                              0.000968
not-logged-in-f5ac928c4dcd4e9cb619    0.512225
Dynamatt                              0.000929
shocko61                              0.039881
not-logged-in-2159616075e1803d6325    0.001442
kjkavene                              0.000941
justsomenoodles                       0.000556
not-logged-in-f09819078c043556a4da    0.000925
BokGlobule                            0.132212
not-logged-in-14849be8c56260e26d09    0.004339
Jessica_Korkmaz                       0.007949
not-logged-in-5f30b2bb9b7f9d49da41    0.014287
not-logged-in-dddbb97cfd84354416f5    0.001233
not-logged-in-4c6d97421fdb5b2fcda3    0.007939
acapirala                             0.079881
ShirleyL                              0.000660
Lavadude                              0.003763
ts135992                              0.000837
not-logged-in-8700ff8251e0704f745c    0.000658
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())